## `Montando apresentação`

`Considerações: `
- O time de logística não considera que o atraso na entrega é um problema relevante e falou que, em média, as entregas estão sendo feitas 10 dias antes do prazo combinado.

- Não é desejado a previsão de uma entrega atrasada, apenas a exposição que esse é um problema que pode impactar os clientes.

- Não queremos a abordagem de "nenhuma entrega pode atrasar". Vamos ser mais tranquilos e seguir na linha de "uma entrega pode atrasar. Como posso melhorar a experiencia do cliente caso isso aconteça?"

### `Preparando o ambiente`

In [87]:
# Importando o sqlite3
import sqlite3

In [88]:
# Importando o pandas
import pandas as pd

In [89]:
# criando a conexão
con =sqlite3.connect('vendas.db')

In [90]:
# criando o cursor
cur = con.cursor()

In [91]:
# utilizando a função já criada
def executa_consulta(consulta):
    resultado = cur.execute(consulta).fetchall()
    resultado = pd.DataFrame(resultado)
    colunas = [i[0] for i in cur.description]
    if resultado.shape[1] > 0:
        resultado.columns = colunas
    print(resultado.shape)
    display(resultado.head(3))
    return resultado

In [92]:
# verificando as tabelas presentes no bando de dados
executa_consulta('SELECT * FROM sqlite_schema WHERE type = "table"')

(9, 5)


,type,name,tbl_name,rootpage,sql
0,table,customers,customers,5018,"CREATE TABLE ""customers"" (\n""index"" INTEGER,\n..."
1,table,geolocation,geolocation,7660,"CREATE TABLE ""geolocation"" (\n""index"" INTEGER,..."
2,table,order_items,order_items,22586,"CREATE TABLE ""order_items"" (\n""index"" INTEGER,..."


,type,name,tbl_name,rootpage,sql
0,table,customers,customers,5018,"CREATE TABLE ""customers"" (\n""index"" INTEGER,\n..."
1,table,geolocation,geolocation,7660,"CREATE TABLE ""geolocation"" (\n""index"" INTEGER,..."
2,table,order_items,order_items,22586,"CREATE TABLE ""order_items"" (\n""index"" INTEGER,..."
3,table,order_payments,order_payments,27121,"CREATE TABLE ""order_payments"" (\n""index"" INTEG..."
4,table,order_reviews,order_reviews,29140,"CREATE TABLE ""order_reviews"" (\n""index"" INTEGE..."
5,table,products,products,33215,"CREATE TABLE ""products"" (\n""index"" INTEGER,\n ..."
6,table,sellers,sellers,33909,"CREATE TABLE ""sellers"" (\n""index"" INTEGER,\n ..."
7,table,product_category_name_translation,product_category_name_translation,33966,"CREATE TABLE ""product_category_name_translatio..."
8,table,orders,orders,2,"CREATE TABLE ""orders"" (\n""index"" INTEGER,\n ""..."


`1.Verificando se o atraso é um problema real`

In [93]:
# visualizando a tabela de pedidos
pedidos = executa_consulta('SELECT * FROM orders')

(99441, 9)


,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


In [94]:
# selecionando apenas as colunas de interesse
pedidos = executa_consulta('SELECT order_id, \
                           order_purchase_timestamp, \
                           order_delivered_customer_date, \
                           order_estimated_delivery_date \
                           FROM orders')

(99441, 4)


,order_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00


In [95]:
# verificando as informações da base
pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   order_purchase_timestamp       99441 non-null  object
 2   order_delivered_customer_date  96476 non-null  object
 3   order_estimated_delivery_date  99441 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


In [99]:
# visualisando os itens nulos em order_delivered_customer_date

pedidos[pedidos['order_delivered_customer_date'].isnull()]

,order_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,2017-04-11 12:22:08,NaT,2017-05-09
44,ee64d42b8cf066f35eac1cf57de1aa85,2018-06-04 16:44:48,NaT,2018-06-28
103,0760a852e4e9d89eb77bf631eaaf1c84,2018-08-03 17:44:42,NaT,2018-08-21
128,15bed8e2fec7fdbadb186b57c46c92f2,2017-09-03 14:22:03,NaT,2017-10-03
154,6942b8da583c2f9957e990d028607019,2018-01-10 11:33:07,NaT,2018-02-07
...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,2018-08-31 16:13:44,NaT,2018-10-01
99313,e9e64a17afa9653aacf2616d94c005b8,2018-01-05 23:07:24,NaT,2018-02-06
99347,a89abace0dcc01eeb267a9660b5ac126,2018-09-06 18:45:47,NaT,2018-09-27
99348,a69ba794cc7deb415c3e15a0a3877e69,2017-08-23 16:28:04,NaT,2017-09-15


In [97]:
# fazendo o tratamento para as colunas de datas

datas = ['order_purchase_timestamp','order_delivered_customer_date','order_estimated_delivery_date']

for i in datas:
    pedidos[i] = pd.to_datetime(pedidos[i], format='%Y-%m-%d %H:%M:%S')

In [98]:
# verificando se foi alterado as datas
pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   order_purchase_timestamp       99441 non-null  datetime64[ns]
 2   order_delivered_customer_date  96476 non-null  datetime64[ns]
 3   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](3), object(1)
memory usage: 3.0+ MB
